#### Подготовка датасета к input формату

In [54]:
import warnings
warnings.filterwarnings("ignore")
import re

Импортируем написанный класс

In [55]:
from create_dataset_m2t import input_json_path, Code2TestPrepareDataset

Вызываем его инициализацию

In [56]:
dataset_m2t = Code2TestPrepareDataset(input_json_path)

Время инициализации датасета: 11.648 секунды


Готовим датасет

In [57]:
dataset_m2t.prepare_dataset()

100%|██████████| 281060/281060 [00:06<00:00, 46051.65it/s]


Время проверки формата запросов: 6.105 секунды


100%|██████████| 281060/281060 [00:00<00:00, 3775343.74it/s]


Время извлечения глобальных признаков из текста кода: 5.675 секунды


100%|██████████| 281060/281060 [00:00<00:00, 369962.73it/s]


Время извлечения комментариев и описаний из текста кода: 9.335 секунды


100%|██████████| 281060/281060 [00:05<00:00, 48869.05it/s]


Время очистки текста кода: 17.924 секунды


100%|██████████| 281060/281060 [00:02<00:00, 116298.94it/s]

Время получения ast-деревьев на основе текста кода: 114.643 секунды
Время подготовки датасета: 153.683 секунды


Полученный датасет:

In [198]:
dataset = dataset_m2t.get_dataset()

In [127]:
dataset.head()

,id,query,response,code_features,LANG_TOKEN,focal_method,focal_cls,callee,focal_method_docs,focal_method_comments,focal_cls_docs,focal_cls_comments,callee_docs,callee_comments,focal_method_ast,focal_cls_ast,callee_ast
0,1,You are a professional python software enginee...,"from microdot import Microdot, Response, abort...","(python, ```python\ndef get(self, key, default...",python,"def get(self, key, default=None):\n kl ...","\nclass NoCaseDict(dict):\n def get(self, k...",No callee examples provided.,,,,,,,Module(\n body=[\n FunctionDef(\n ...,Module(\n body=[\n ClassDef(\n ...,AST_TOKEN
1,2,You are a professional python software enginee...,"from microdot import Microdot, Response, abort...","(python, ```python\ndef get(self, url_pattern)...",python,"def get(self, url_pattern):\n """"""Decora...","\nclass Microdot:\n def route(self, url_pat...",No callee examples provided.,Decorator that is used to register a function ...,# ...,Decorator that is used to register a function ...,# ...,,,Module(\n body=[\n FunctionDef(\n ...,Module(\n body=[\n ClassDef(\n ...,AST_TOKEN
2,3,You are a professional python software enginee...,"from microdot import Microdot, Response, abort...","(python, ```python\ndef post(self, url_pattern...",python,"def post(self, url_pattern):\n """"""Decor...","\nclass Microdot:\n def route(self, url_pat...",No callee examples provided.,Decorator that is used to register a function ...,# ...,Decorator that is used to register a function ...,# ...,,,Module(\n body=[\n FunctionDef(\n ...,Module(\n body=[\n ClassDef(\n ...,AST_TOKEN
3,4,You are a professional python software enginee...,"from microdot import Microdot, Response, abort...","(python, ```python\ndef mount(self, subapp, ur...",python,"def mount(self, subapp, url_prefix=''):\n ...","\nclass Microdot:\n def mount(self, subapp,...",No callee examples provided.,"Mount a sub-application, optionally under the ...",,"Mount a sub-application, optionally under the ...",,,,Module(\n body=[\n FunctionDef(\n ...,Module(\n body=[\n ClassDef(\n ...,AST_TOKEN
4,5,You are a professional python software enginee...,from pyner.named_entity.corpus import bio2bioe...,"(python, ```python\ndef iob2bio(tags):\n pr...",python,def iob2bio(tags):\n processed_tags = [] #...,"\ndef split_tag(tag: str):\n """"""\n Split...",No callee examples provided.,,# should be bio format\n# case1. I-ORG I-ORG\n...,\n Split tag into state and named entity ca...,# should be bio format\n# case1. I-ORG I-ORG\n...,,,Module(\n body=[\n FunctionDef(\n ...,Module(\n body=[\n FunctionDef(\n ...,AST_TOKEN


Исследуем количество calee, для которых не существует примеров вызовов функции

In [128]:
print(f"Количество callee без вызовов: {len(dataset[dataset['callee'] == 'No callee examples provided.'])}")
print(f"Количество пар всего: {dataset.shape[0]}")
print(f"Доля отсутсвия экземпляров вызовов: {len(dataset[dataset['callee'] == 'No callee examples provided.']) / dataset.shape[0]:.3f}")

Количество callee без вызовов: 255967
Количество пар всего: 281060
Доля отсутсвия экземпляров вызовов: 0.911


Заметим, что почти 92% всех сэмплов не имеют в своем составе примеров вызова. 

Отсюда есть смысл выдвинуть следующий подход: необходимо включить имеющиеся примеры в состав каких-либо колонок или попросту убрать данную колонку. Вероятно, подобное смещение выборки засчет добавления дополнительной (что более важно, присущей небольшой части данных) информации может запутать модель. Поэтому на текцщем этапе решено отбросить данную колонку и поработать с остальными для генерации input формата данных

Исследуем "непустые" значения в колонке callee:

In [129]:
dataset = dataset.drop(columns=['callee', 'callee_docs', 'callee_comments', 'callee_ast'])

Обновленный формат датасета:

In [130]:
dataset.head()

,id,query,response,code_features,LANG_TOKEN,focal_method,focal_cls,focal_method_docs,focal_method_comments,focal_cls_docs,focal_cls_comments,focal_method_ast,focal_cls_ast
0,1,You are a professional python software enginee...,"from microdot import Microdot, Response, abort...","(python, ```python\ndef get(self, key, default...",python,"def get(self, key, default=None):\n kl ...","\nclass NoCaseDict(dict):\n def get(self, k...",,,,,Module(\n body=[\n FunctionDef(\n ...,Module(\n body=[\n ClassDef(\n ...
1,2,You are a professional python software enginee...,"from microdot import Microdot, Response, abort...","(python, ```python\ndef get(self, url_pattern)...",python,"def get(self, url_pattern):\n """"""Decora...","\nclass Microdot:\n def route(self, url_pat...",Decorator that is used to register a function ...,# ...,Decorator that is used to register a function ...,# ...,Module(\n body=[\n FunctionDef(\n ...,Module(\n body=[\n ClassDef(\n ...
2,3,You are a professional python software enginee...,"from microdot import Microdot, Response, abort...","(python, ```python\ndef post(self, url_pattern...",python,"def post(self, url_pattern):\n """"""Decor...","\nclass Microdot:\n def route(self, url_pat...",Decorator that is used to register a function ...,# ...,Decorator that is used to register a function ...,# ...,Module(\n body=[\n FunctionDef(\n ...,Module(\n body=[\n ClassDef(\n ...
3,4,You are a professional python software enginee...,"from microdot import Microdot, Response, abort...","(python, ```python\ndef mount(self, subapp, ur...",python,"def mount(self, subapp, url_prefix=''):\n ...","\nclass Microdot:\n def mount(self, subapp,...","Mount a sub-application, optionally under the ...",,"Mount a sub-application, optionally under the ...",,Module(\n body=[\n FunctionDef(\n ...,Module(\n body=[\n ClassDef(\n ...
4,5,You are a professional python software enginee...,from pyner.named_entity.corpus import bio2bioe...,"(python, ```python\ndef iob2bio(tags):\n pr...",python,def iob2bio(tags):\n processed_tags = [] #...,"\ndef split_tag(tag: str):\n """"""\n Split...",,# should be bio format\n# case1. I-ORG I-ORG\n...,\n Split tag into state and named entity ca...,# should be bio format\n# case1. I-ORG I-ORG\n...,Module(\n body=[\n FunctionDef(\n ...,Module(\n body=[\n FunctionDef(\n ...


Далее исследуем наличие описаний и комментариев в рамках *focal_method*

In [131]:
print(f"Количество пустых строк в комментариях для focal_method: {len(dataset[dataset['focal_method_comments'] == ''])}")
print(f"Количество пустых строк в описаниях для focal_method: {len(dataset[dataset['focal_method_docs'] == ''])}")
print(f"Доля пустых строк в комментариях для focal_method: {len(dataset[dataset['focal_method_comments'] == ''])/dataset.shape[0]:.3f}")
print(f"Доля пустых строк в описаниях для focal_method: {len(dataset[dataset['focal_method_docs'] == ''])/dataset.shape[0]:.3f}")

Количество пустых строк в комментариях для focal_method: 206473
Количество пустых строк в описаниях для focal_method: 92364
Доля пустых строк в комментариях для focal_method: 0.735
Доля пустых строк в описаниях для focal_method: 0.329


Доли не такие значительные для focal_method. Следовательно, необходимо объединить информацию об описаниях и комментариях, например, создав общий комментарий к коду на естественном языке, позволив модели обрабатывать и кодовую, и дополнительную, представленную на языке человека информацию (пусть, на английском языке). Имеющиеся пустые предстваления заменим на <MASK> токен (в какой-то степени регуляризация модели)

In [132]:
print(dataset['focal_method'].values[1])

def get(self, url_pattern):
        """Decorator that is used to register a function as a ``GET`` request
        handler for a given URL.

        :param url_pattern: The URL pattern that will be compared against
                            incoming requests.

        This decorator can be used as an alias to the ``route`` decorator with
        ``methods=['GET']``.

        Example::

            @app.get('/users/<int:id>')
            def get_user(request, id):
                # ...
        """
        return self.route(url_pattern, methods=['GET'])



Из текста кода необходимо убрать все описания и комментарии

In [133]:
def remove_docs(row, col_name, docs_col_name):
    '''Функция удаления описания из указанной колонки'''
    return row[col_name].replace(row[docs_col_name], '')

def remove_three_quotes(row, col_name):
    '''Функция удаления """""" из указанной колонки'''
    return row[col_name].replace('""""""', '')

def remove_three_quotes_v2(row, col_name):
    '''Функция удаления """""" из указанной колонки'''
    return row[col_name].replace('''''''', '')

def drop_ln(row, col_name):
    '''Функция удаления \n из указанной колонки'''
    return row[col_name].replace('\n', '')

def drop_large_spaces(row, col_name):
    '''Функция удаления длинных пробелов (\t) из указанной колонки'''
    cleaned_string = re.sub(r'\s+', ' ', row[col_name])
    return cleaned_string

def remove_comment(row, col_name, comments_col_name):
    '''Функция удаления комментариев из указанной колонки'''
    return row[col_name].replace(row[comments_col_name], '')

# Пример использования функций для колонки 'focal_method'
dataset['focal_method'] = dataset.progress_apply(remove_docs, axis=1, args=('focal_method', 'focal_method_docs'))
dataset['focal_method'] = dataset.progress_apply(remove_three_quotes, axis=1, args=('focal_method',))
dataset['focal_method'] = dataset.progress_apply(remove_three_quotes_v2, axis=1, args=('focal_method',))
dataset['focal_method'] = dataset.progress_apply(drop_ln, axis=1, args=('focal_method',))
dataset['focal_method'] = dataset.progress_apply(drop_large_spaces, axis=1, args=('focal_method',))
dataset['focal_method'] = dataset.progress_apply(remove_comment, axis=1, args=('focal_method', 'focal_method_comments'))

100%|██████████| 281060/281060 [00:00<00:00, 360333.90it/s]


In [134]:
print(dataset['focal_method'].values[1])

def get(self, url_pattern): return self.route(url_pattern, methods=['GET'])


Далее работаем над получением общего вида описаний и комментариев для *focal_method*

In [135]:
def drop_hashtag_symbol(row, col_name):
    '''Функция отбрасывает символ # для указанной колонки'''
    return row[col_name].replace('#', '')

def remove_non_language_symbols(row, col_name):
    '''Функция удаления неалфавитных символов из указанной колонки'''
    return re.sub(r'[^a-zA-Z0-9\s]', '', row[col_name])

def strip_text(row, col_name):
    '''Функция удаления пробелов в начале и в конце строки для указанной колонки'''
    return row[col_name].strip()

# Пример использования функций для колонки 'focal_method_comments'
dataset['focal_method_comments'] = dataset.progress_apply(drop_hashtag_symbol, axis=1, args=('focal_method_comments',))
dataset['focal_method_comments'] = dataset.progress_apply(drop_ln, axis=1, args=('focal_method_comments',))
dataset['focal_method_comments'] = dataset.progress_apply(drop_large_spaces, axis=1, args=('focal_method_comments',))
dataset['focal_method_comments'] = dataset.progress_apply(remove_non_language_symbols, axis=1, args=('focal_method_comments',))
dataset['focal_method_comments'] = dataset.progress_apply(strip_text, axis=1, args=('focal_method_comments',))


100%|██████████| 281060/281060 [00:00<00:00, 441543.88it/s]


Теперь мы можем создать общее представление естественного языка для нашей функции из focal_method

In [136]:
def create_NL_code_representation(row, desc_col, comm_col):
    '''Функция создания описания'''
    result = "DESCRIPTION_TOKEN: {} COMMENTS_TOKEN: {}".format(row[desc_col], row[comm_col])
    return result

def replace_empty_info(row, info_col):
    '''Функция для определения пустого описания'''
    return row[info_col].replace("DESCRIPTION_TOKEN:  COMMENTS_TOKEN: ", "INFO_TOKEN")

def replace_INFO_TOKEN_non_empty(row, info_col):
    '''Функция замены INFO_TOKEN при условии не равенства строки INFO_TOKEN'''
    if row[info_col] == 'INFO_TOKEN':
        return 'INFO_TOKEN'
    return row[info_col].replace('INFO_TOKEN', 'DESCRIPTION_TOKEN: ')

# Пример использования функций для колонки 'focal_method_info'
dataset['focal_method_info'] = dataset.progress_apply(
    create_NL_code_representation, axis=1, args=('focal_method_docs', 'focal_method_comments')
)
dataset['focal_method_info'] = dataset.progress_apply(
    replace_empty_info, axis=1, args=('focal_method_info',)
)
dataset['focal_method_info'] = dataset.progress_apply(
    replace_INFO_TOKEN_non_empty, axis=1, args=('focal_method_info',)
)

100%|██████████| 281060/281060 [00:00<00:00, 363234.62it/s]


In [137]:
dataset['focal_method_info'] = dataset.progress_apply(drop_ln, axis=1, args=('focal_method_info',))

100%|██████████| 281060/281060 [00:00<00:00, 327593.40it/s]


In [138]:
dataset['focal_method_info'] = dataset.progress_apply(drop_large_spaces, axis=1, args=('focal_method_info',))

100%|██████████| 281060/281060 [00:03<00:00, 71108.38it/s] 


In [141]:
dataset['focal_method_info'].values[0]

'INFO_TOKEN'

In [142]:
dataset.head()

,id,query,response,code_features,LANG_TOKEN,focal_method,focal_cls,focal_method_docs,focal_method_comments,focal_cls_docs,focal_cls_comments,focal_method_ast,focal_cls_ast,focal_method_info
0,1,You are a professional python software enginee...,"from microdot import Microdot, Response, abort...","(python, ```python\ndef get(self, key, default...",python,"def get(self, key, default=None): kl = key.low...","\nclass NoCaseDict(dict):\n def get(self, k...",,,,,Module(\n body=[\n FunctionDef(\n ...,Module(\n body=[\n ClassDef(\n ...,INFO_TOKEN
1,2,You are a professional python software enginee...,"from microdot import Microdot, Response, abort...","(python, ```python\ndef get(self, url_pattern)...",python,"def get(self, url_pattern): return self.route(...","\nclass Microdot:\n def route(self, url_pat...",Decorator that is used to register a function ...,,Decorator that is used to register a function ...,# ...,Module(\n body=[\n FunctionDef(\n ...,Module(\n body=[\n ClassDef(\n ...,DESCRIPTION_TOKEN: Decorator that is used to r...
2,3,You are a professional python software enginee...,"from microdot import Microdot, Response, abort...","(python, ```python\ndef post(self, url_pattern...",python,"def post(self, url_pattern): return self.route...","\nclass Microdot:\n def route(self, url_pat...",Decorator that is used to register a function ...,,Decorator that is used to register a function ...,# ...,Module(\n body=[\n FunctionDef(\n ...,Module(\n body=[\n ClassDef(\n ...,DESCRIPTION_TOKEN: Decorator that is used to r...
3,4,You are a professional python software enginee...,"from microdot import Microdot, Response, abort...","(python, ```python\ndef mount(self, subapp, ur...",python,"def mount(self, subapp, url_prefix=''): for me...","\nclass Microdot:\n def mount(self, subapp,...","Mount a sub-application, optionally under the ...",,"Mount a sub-application, optionally under the ...",,Module(\n body=[\n FunctionDef(\n ...,Module(\n body=[\n ClassDef(\n ...,"DESCRIPTION_TOKEN: Mount a sub-application, op..."
4,5,You are a professional python software enginee...,from pyner.named_entity.corpus import bio2bioe...,"(python, ```python\ndef iob2bio(tags):\n pr...",python,def iob2bio(tags): processed_tags = [] # shoul...,"\ndef split_tag(tag: str):\n """"""\n Split...",,should be bio format case1 IORG IORG case2 IO...,\n Split tag into state and named entity ca...,# should be bio format\n# case1. I-ORG I-ORG\n...,Module(\n body=[\n FunctionDef(\n ...,Module(\n body=[\n FunctionDef(\n ...,DESCRIPTION_TOKEN: should be bio format case1 ...


Посмотрим на количество "непустой" информации о функции *focal_method*

In [143]:
print(f"Количество пустой информации о функциях из focal_method: {len(dataset[dataset['focal_method_info'] == 'INFO_TOKEN'])}")
print(f"Доля пустой информации о функциях из focal_method: {len(dataset[dataset['focal_method_info'] == 'INFO_TOKEN']) / dataset.shape[0]:.3f}")

Количество пустой информации о функциях из focal_method: 75586
Доля пустой информации о функциях из focal_method: 0.269


Таким образом, доля комментариев с пустой информацией составляет примерно четверть. Поэтому сохраним информацию на естественном языке и будем использовать её при обучении модели

Далее исследуем наличие ast-представлений для *focal_method*

In [144]:
print(f"Количество необработанных AST-представлений кода {len(dataset[dataset['focal_method_ast'] == 'AST_TOKEN'])}")
print(f"Доля необработанных AST-представлений кода {len(dataset[dataset['focal_method_ast'] == 'AST_TOKEN']) / dataset.shape[0]:.3f}")

Количество необработанных AST-представлений кода 602
Доля необработанных AST-представлений кода 0.002


Таких предствалений немного, поэтому, чтобы не путать модель, отбросим эти представления:

In [145]:
dataset = dataset[dataset['focal_method_ast'] != 'AST_TOKEN']
dataset.shape

(280458, 14)

In [147]:
dataset.head()

,id,query,response,code_features,LANG_TOKEN,focal_method,focal_cls,focal_method_docs,focal_method_comments,focal_cls_docs,focal_cls_comments,focal_method_ast,focal_cls_ast,focal_method_info
0,1,You are a professional python software enginee...,"from microdot import Microdot, Response, abort...","(python, ```python\ndef get(self, key, default...",python,"def get(self, key, default=None): kl = key.low...","\nclass NoCaseDict(dict):\n def get(self, k...",,,,,Module(\n body=[\n FunctionDef(\n ...,Module(\n body=[\n ClassDef(\n ...,INFO_TOKEN
1,2,You are a professional python software enginee...,"from microdot import Microdot, Response, abort...","(python, ```python\ndef get(self, url_pattern)...",python,"def get(self, url_pattern): return self.route(...","\nclass Microdot:\n def route(self, url_pat...",Decorator that is used to register a function ...,,Decorator that is used to register a function ...,# ...,Module(\n body=[\n FunctionDef(\n ...,Module(\n body=[\n ClassDef(\n ...,DESCRIPTION_TOKEN: Decorator that is used to r...
2,3,You are a professional python software enginee...,"from microdot import Microdot, Response, abort...","(python, ```python\ndef post(self, url_pattern...",python,"def post(self, url_pattern): return self.route...","\nclass Microdot:\n def route(self, url_pat...",Decorator that is used to register a function ...,,Decorator that is used to register a function ...,# ...,Module(\n body=[\n FunctionDef(\n ...,Module(\n body=[\n ClassDef(\n ...,DESCRIPTION_TOKEN: Decorator that is used to r...
3,4,You are a professional python software enginee...,"from microdot import Microdot, Response, abort...","(python, ```python\ndef mount(self, subapp, ur...",python,"def mount(self, subapp, url_prefix=''): for me...","\nclass Microdot:\n def mount(self, subapp,...","Mount a sub-application, optionally under the ...",,"Mount a sub-application, optionally under the ...",,Module(\n body=[\n FunctionDef(\n ...,Module(\n body=[\n ClassDef(\n ...,"DESCRIPTION_TOKEN: Mount a sub-application, op..."
4,5,You are a professional python software enginee...,from pyner.named_entity.corpus import bio2bioe...,"(python, ```python\ndef iob2bio(tags):\n pr...",python,def iob2bio(tags): processed_tags = [] # shoul...,"\ndef split_tag(tag: str):\n """"""\n Split...",,should be bio format case1 IORG IORG case2 IO...,\n Split tag into state and named entity ca...,# should be bio format\n# case1. I-ORG I-ORG\n...,Module(\n body=[\n FunctionDef(\n ...,Module(\n body=[\n FunctionDef(\n ...,DESCRIPTION_TOKEN: should be bio format case1 ...


Уберем лишние пробелы и перенос строки для ast-представлений

In [148]:
dataset['focal_method_ast'] = dataset.progress_apply(drop_ln, axis = 1, args = ['focal_method_ast'])
dataset['focal_method_ast'] = dataset.progress_apply(drop_large_spaces, axis = 1, args = ['focal_method_ast'])

100%|██████████| 280458/280458 [00:13<00:00, 21206.99it/s]


In [149]:
print(dataset['focal_method_ast'].values[0])

Module( body=[ FunctionDef( name='get', args=arguments( posonlyargs=[], args=[ arg(arg='self'), arg(arg='key'), arg(arg='default')], kwonlyargs=[], kw_defaults=[], defaults=[ Constant(value=None)]), body=[ Assign( targets=[ Name(id='kl', ctx=Store())], value=Call( func=Attribute( value=Name(id='key', ctx=Load()), attr='lower', ctx=Load()), args=[], keywords=[])), Return( value=Call( func=Attribute( value=Call( func=Name(id='super', ctx=Load()), args=[], keywords=[]), attr='get', ctx=Load()), args=[ Call( func=Attribute( value=Attribute( value=Name(id='self', ctx=Load()), attr='keymap', ctx=Load()), attr='get', ctx=Load()), args=[ Name(id='kl', ctx=Load()), Name(id='kl', ctx=Load())], keywords=[]), Name(id='default', ctx=Load())], keywords=[]))], decorator_list=[], type_params=[])], type_ignores=[])


In [150]:
dataset = dataset.drop(columns=['focal_method_docs', 
                                'focal_method_comments'])

In [86]:
dataset.head()

,id,query,response,code_features,LANG_TOKEN,focal_method,focal_cls,focal_cls_docs,focal_cls_comments,focal_method_ast,focal_cls_ast,focal_method_info
0,1,You are a professional python software enginee...,"from microdot import Microdot, Response, abort...","(python, ```python\ndef get(self, key, default...",python,"def get(self, key, default=None): kl = key.low...","\nclass NoCaseDict(dict):\n def get(self, k...",,,"Module( body=[ FunctionDef( name='get', args=a...",Module(\n body=[\n ClassDef(\n ...,INFO_TOKEN
1,2,You are a professional python software enginee...,"from microdot import Microdot, Response, abort...","(python, ```python\ndef get(self, url_pattern)...",python,"def get(self, url_pattern): return self.route(...","\nclass Microdot:\n def route(self, url_pat...",Decorator that is used to register a function ...,# ...,"Module( body=[ FunctionDef( name='get', args=a...",Module(\n body=[\n ClassDef(\n ...,DESCRIPTION_TOKEN: Decorator that is used to r...
2,3,You are a professional python software enginee...,"from microdot import Microdot, Response, abort...","(python, ```python\ndef post(self, url_pattern...",python,"def post(self, url_pattern): return self.route...","\nclass Microdot:\n def route(self, url_pat...",Decorator that is used to register a function ...,# ...,"Module( body=[ FunctionDef( name='post', args=...",Module(\n body=[\n ClassDef(\n ...,DESCRIPTION_TOKEN: Decorator that is used to r...
3,4,You are a professional python software enginee...,"from microdot import Microdot, Response, abort...","(python, ```python\ndef mount(self, subapp, ur...",python,"def mount(self, subapp, url_prefix=''): for me...","\nclass Microdot:\n def mount(self, subapp,...","Mount a sub-application, optionally under the ...",,"Module( body=[ FunctionDef( name='mount', args...",Module(\n body=[\n ClassDef(\n ...,"DESCRIPTION_TOKEN: Mount a sub-application, op..."
4,5,You are a professional python software enginee...,from pyner.named_entity.corpus import bio2bioe...,"(python, ```python\ndef iob2bio(tags):\n pr...",python,def iob2bio(tags): processed_tags = [] # shoul...,"\ndef split_tag(tag: str):\n """"""\n Split...",\n Split tag into state and named entity ca...,# should be bio format\n# case1. I-ORG I-ORG\n...,"Module( body=[ FunctionDef( name='iob2bio', ar...",Module(\n body=[\n FunctionDef(\n ...,DESCRIPTION_TOKEN: # should be bio format\n# c...


Далее необходимо сделать аналогичные действия и подготовить *focal_cls* к input формату

In [152]:
print(dataset['focal_cls'].values[1])


class Microdot:
    def route(self, url_pattern, methods=None):
        """Decorator that is used to register a function as a request handler
        for a given URL.
        :param url_pattern: The URL pattern that will be compared against
                            incoming requests.
        :param methods: The list of HTTP methods to be handled by the
                        decorated function. If omitted, only ``GET`` requests
                        are handled.
        The URL pattern can be a static path (for example, ``/users`` or
        ``/api/invoices/search``) or a path with dynamic components enclosed
        in ``<`` and ``>`` (for example, ``/users/<id>`` or
        ``/invoices/<number>/products``). Dynamic path components can also
        include a type prefix, separated from the name with a colon (for
        example, ``/users/<int:id>``). The type can be ``string`` (the
        default), ``int``, ``path`` or ``re:[regular-expression]``.
        The first argument of

In [153]:
def remove_triple_quotes_content(row, col_name):
    '''Функция для удаления содержимого внутри тройных кавычек'''
    # Регулярное выражение для поиска содержимого внутри тройных кавычек
    pattern = re.compile(r'""".*?"""', re.DOTALL)
    # Замена содержимого внутри тройных кавычек на пустую строку
    return pattern.sub('', row[col_name])


dataset['focal_cls'] = dataset.progress_apply(remove_triple_quotes_content, axis = 1, args = ['focal_cls'])
dataset['focal_cls'] = dataset.progress_apply(remove_docs, axis=1, args=('focal_cls', 'focal_cls_docs'))
dataset['focal_cls'] = dataset.progress_apply(remove_three_quotes, axis=1, args=('focal_cls',))
dataset['focal_cls'] = dataset.progress_apply(remove_three_quotes_v2, axis=1, args=('focal_cls',))
dataset['focal_cls'] = dataset.progress_apply(drop_ln, axis=1, args=('focal_cls',))
dataset['focal_cls'] = dataset.progress_apply(drop_large_spaces, axis=1, args=('focal_cls',))
dataset['focal_cls'] = dataset.progress_apply(remove_comment, axis=1, args=('focal_cls', 'focal_cls_comments'))

100%|██████████| 280458/280458 [00:00<00:00, 325196.94it/s]


In [155]:
print(dataset['focal_cls'].values[1])

class Microdot: def route(self, url_pattern, methods=None): def decorated(f): self.url_map.append( ([m.upper() for m in (methods or ['GET'])], URLPattern(url_pattern), f)) return f return decorated def get(self, url_pattern): return self.route(url_pattern, methods=['GET'])


Теперь необходимо заменить название функции внутри *focal_class*, соответствующее *focal_method* на специальный токен <FUNC_TOKEN>, чтобы не учитывать внутри него информацию о *focal_method*, а работать исключительно с окружением функции, для которой мы пишем тесты

In [91]:
def replace_func(row, column_fm, column_fc):
    '''Функция будет заменять содержимое одной колонки внутри другой'''
    target = row[column_fc]
    substring = row[column_fm]
    return target.replace(substring, 'FUNC_TOKEN')

def insert_space(row, column, token):
    '''Функция вставить пробел дополнительный пробел для токена в колонке'''
    return row[column].replace(token, f" {token} ").strip().replace('  ', ' ')

dataset['focal_cls'] = dataset.progress_apply(replace_func, axis=1, args=('focal_method', 'focal_cls'))
dataset['focal_cls'] = dataset.progress_apply(insert_space, axis=1, args=('focal_cls', 'FUNC_TOKEN'))

100%|██████████| 280458/280458 [00:00<00:00, 286257.19it/s]


In [151]:
print(dataset['focal_cls'].values[1])


class Microdot:
    def route(self, url_pattern, methods=None):
        """Decorator that is used to register a function as a request handler
        for a given URL.
        :param url_pattern: The URL pattern that will be compared against
                            incoming requests.
        :param methods: The list of HTTP methods to be handled by the
                        decorated function. If omitted, only ``GET`` requests
                        are handled.
        The URL pattern can be a static path (for example, ``/users`` or
        ``/api/invoices/search``) or a path with dynamic components enclosed
        in ``<`` and ``>`` (for example, ``/users/<id>`` or
        ``/invoices/<number>/products``). Dynamic path components can also
        include a type prefix, separated from the name with a colon (for
        example, ``/users/<int:id>``). The type can be ``string`` (the
        default), ``int``, ``path`` or ``re:[regular-expression]``.
        The first argument of

Приводим к корректному виду описание *focal_cls* на естественном языке

In [156]:
dataset['focal_cls_comments'] = dataset.progress_apply(drop_hashtag_symbol, axis=1, args=('focal_cls_comments',))
dataset['focal_cls_comments'] = dataset.progress_apply(drop_ln, axis=1, args=('focal_cls_comments',))
dataset['focal_cls_comments'] = dataset.progress_apply(drop_large_spaces, axis=1, args=('focal_cls_comments',))
dataset['focal_cls_comments'] = dataset.progress_apply(remove_non_language_symbols, axis=1, args=('focal_cls_comments',))
dataset['focal_cls_comments'] = dataset.progress_apply(strip_text, axis=1, args=('focal_cls_comments',))

100%|██████████| 280458/280458 [00:00<00:00, 482367.35it/s]


In [157]:
print(dataset['focal_cls_docs'].values[4])


    Split tag into state and named entity category.
    Parameters
    ---
    tag (str)
        NE tag (e.g. B-PER)
    


In [159]:
dataset['focal_cls_info'] = dataset.progress_apply(
    create_NL_code_representation, axis=1, args=('focal_cls_docs', 'focal_cls_comments')
)
dataset['focal_cls_info'] = dataset.progress_apply(
    replace_empty_info, axis=1, args=('focal_cls_info',)
)
dataset['focal_cls_info'] = dataset.progress_apply(
    replace_INFO_TOKEN_non_empty, axis=1, args=('focal_cls_info',)
)

100%|██████████| 280458/280458 [00:00<00:00, 356763.65it/s]


In [160]:
dataset['focal_method_info'].values[1]

"DESCRIPTION_TOKEN: Decorator that is used to register a function as a ``GET`` request handler for a given URL. :param url_pattern: The URL pattern that will be compared against incoming requests. This decorator can be used as an alias to the ``route`` decorator with ``methods=['GET']``. Example:: @app.get('/users/<int:id>') def get_user(request, id): # ... COMMENTS_TOKEN: "

In [169]:
dataset['focal_cls_info'] = dataset.progress_apply(drop_ln, axis=1, args=('focal_cls_info',))
dataset['focal_cls_info'] = dataset.progress_apply(drop_large_spaces, axis=1, args=('focal_cls_info',))

100%|██████████| 280458/280458 [00:02<00:00, 95958.07it/s] 


In [172]:
print(dataset['focal_cls_info'].values[4])

DESCRIPTION_TOKEN: Split tag into state and named entity category. Parameters --- tag (str) NE tag (e.g. B-PER) COMMENTS_TOKEN: should be bio format case1 IORG IORG case2 IORG IPERSON case3 O IORG case4 IORG IORG


In [ ]:
dataset['focal_cls_ast'] = dataset.progress_apply(drop_ln, axis = 1, args = ['focal_cls_ast'])
dataset['focal_cls_ast'] = dataset.progress_apply(drop_large_spaces, axis = 1, args = ['focal_cls_ast'])

100%|██████████| 280458/280458 [00:08<00:00, 33157.47it/s]


In [177]:
dataset['focal_cls_ast'].values[0]

"Module( body=[ FunctionDef( name='get', args=arguments( posonlyargs=[], args=[ arg(arg='self'), arg(arg='key'), arg(arg='default')], kwonlyargs=[], kw_defaults=[], defaults=[ Constant(value=None)]), body=[ Assign( targets=[ Name(id='kl', ctx=Store())], value=Call( func=Attribute( value=Name(id='key', ctx=Load()), attr='lower', ctx=Load()), args=[], keywords=[])), Return( value=Call( func=Attribute( value=Call( func=Name(id='super', ctx=Load()), args=[], keywords=[]), attr='get', ctx=Load()), args=[ Call( func=Attribute( value=Attribute( value=Name(id='self', ctx=Load()), attr='keymap', ctx=Load()), attr='get', ctx=Load()), args=[ Name(id='kl', ctx=Load()), Name(id='kl', ctx=Load())], keywords=[]), Name(id='default', ctx=Load())], keywords=[]))], decorator_list=[], type_params=[])], type_ignores=[])"

In [180]:
dataset = dataset.drop(columns = ['focal_cls_comments',
                                  'focal_cls_docs'])

In [181]:
dataset.head()

,id,query,response,code_features,LANG_TOKEN,focal_method,focal_cls,focal_method_ast,focal_cls_ast,focal_method_info,focal_cls_info
0,1,You are a professional python software enginee...,"from microdot import Microdot, Response, abort...","(python, ```python\ndef get(self, key, default...",python,"def get(self, key, default=None): kl = key.low...","class NoCaseDict(dict): def get(self, key, def...","Module( body=[ FunctionDef( name='get', args=a...","Module( body=[ FunctionDef( name='get', args=a...",INFO_TOKEN,INFO_TOKEN
1,2,You are a professional python software enginee...,"from microdot import Microdot, Response, abort...","(python, ```python\ndef get(self, url_pattern)...",python,"def get(self, url_pattern): return self.route(...","class Microdot: def route(self, url_pattern, m...","Module( body=[ FunctionDef( name='get', args=a...","Module( body=[ FunctionDef( name='get', args=a...",DESCRIPTION_TOKEN: Decorator that is used to r...,DESCRIPTION_TOKEN: Decorator that is used to r...
2,3,You are a professional python software enginee...,"from microdot import Microdot, Response, abort...","(python, ```python\ndef post(self, url_pattern...",python,"def post(self, url_pattern): return self.route...","class Microdot: def route(self, url_pattern, m...","Module( body=[ FunctionDef( name='post', args=...","Module( body=[ FunctionDef( name='post', args=...",DESCRIPTION_TOKEN: Decorator that is used to r...,DESCRIPTION_TOKEN: Decorator that is used to r...
3,4,You are a professional python software enginee...,"from microdot import Microdot, Response, abort...","(python, ```python\ndef mount(self, subapp, ur...",python,"def mount(self, subapp, url_prefix=''): for me...","class Microdot: def mount(self, subapp, url_pr...","Module( body=[ FunctionDef( name='mount', args...","Module( body=[ FunctionDef( name='mount', args...","DESCRIPTION_TOKEN: Mount a sub-application, op...","DESCRIPTION_TOKEN: Mount a sub-application, op..."
4,5,You are a professional python software enginee...,from pyner.named_entity.corpus import bio2bioe...,"(python, ```python\ndef iob2bio(tags):\n pr...",python,def iob2bio(tags): processed_tags = [] # shoul...,"def split_tag(tag: str): if tag in [""O"", ""-X-""...","Module( body=[ FunctionDef( name='iob2bio', ar...","Module( body=[ FunctionDef( name='iob2bio', ar...",DESCRIPTION_TOKEN: should be bio format case1 ...,DESCRIPTION_TOKEN: Split tag into state and na...


In [184]:
dataset['focal_method_ast'].values[0] == dataset['focal_cls_ast'].values[0]

True

In [183]:
dataset['focal_cls_ast'].values[0]

"Module( body=[ FunctionDef( name='get', args=arguments( posonlyargs=[], args=[ arg(arg='self'), arg(arg='key'), arg(arg='default')], kwonlyargs=[], kw_defaults=[], defaults=[ Constant(value=None)]), body=[ Assign( targets=[ Name(id='kl', ctx=Store())], value=Call( func=Attribute( value=Name(id='key', ctx=Load()), attr='lower', ctx=Load()), args=[], keywords=[])), Return( value=Call( func=Attribute( value=Call( func=Name(id='super', ctx=Load()), args=[], keywords=[]), attr='get', ctx=Load()), args=[ Call( func=Attribute( value=Attribute( value=Name(id='self', ctx=Load()), attr='keymap', ctx=Load()), attr='get', ctx=Load()), args=[ Name(id='kl', ctx=Load()), Name(id='kl', ctx=Load())], keywords=[]), Name(id='default', ctx=Load())], keywords=[]))], decorator_list=[], type_params=[])], type_ignores=[])"

In [189]:
dataset['focal_method'].values[0]

'def get(self, key, default=None): kl = key.lower() return super().get(self.keymap.get(kl, kl), default)'

In [190]:
dataset['focal_cls'].values[0]

'class NoCaseDict(dict): def get(self, key, default=None): kl = key.lower() return super().get(self.keymap.get(kl, kl), default)'

In [199]:
import ast
# Пытаемся сразу построить AST-дерево
tree = ast.parse(dataset['focal_method'].values[3])
# Преобразуем AST в строку и возвращаем
ast_tree = ast.dump(tree, indent=4)
	

ast_tree

"Module(\n    body=[\n        FunctionDef(\n            name='mount',\n            args=arguments(\n                posonlyargs=[],\n                args=[\n                    arg(arg='self'),\n                    arg(arg='subapp'),\n                    arg(arg='url_prefix')],\n                kwonlyargs=[],\n                kw_defaults=[],\n                defaults=[\n                    Constant(value='')]),\n            body=[\n                Expr(\n                    value=Constant(value='Mount a sub-application, optionally under the given URL prefix.\\n\\n        :param subapp: The sub-application to mount.\\n        :param url_prefix: The URL prefix to mount the application under.\\n        ')),\n                For(\n                    target=Tuple(\n                        elts=[\n                            Name(id='methods', ctx=Store()),\n                            Name(id='pattern', ctx=Store()),\n                            Name(id='handler', ctx=Store())],\n         

In [ ]:
from create_dataset_m2t import Code2TestPrepareDataset

In [200]:
tree = ast.parse(dataset['focal_cls'].values[3])
# Преобразуем AST в строку и возвращаем
ast_tree = ast.dump(tree, indent=4)
	

ast_tree

"Module(\n    body=[\n        ClassDef(\n            name='Microdot',\n            bases=[],\n            keywords=[],\n            body=[\n                FunctionDef(\n                    name='mount',\n                    args=arguments(\n                        posonlyargs=[],\n                        args=[\n                            arg(arg='self'),\n                            arg(arg='subapp'),\n                            arg(arg='url_prefix')],\n                        kwonlyargs=[],\n                        kw_defaults=[],\n                        defaults=[\n                            Constant(value='')]),\n                    body=[\n                        Expr(\n                            value=Constant(value='Mount a sub-application, optionally under the given URL prefix.\\n        :param subapp: The sub-application to mount.\\n        :param url_prefix: The URL prefix to mount the application under.\\n        ')),\n                        For(\n                    

In [201]:
Code2TestPrepareDataset.get_ast_representation(dataset['focal_cls'].values[3])

"Module(\n    body=[\n        ClassDef(\n            name='Microdot',\n            bases=[],\n            keywords=[],\n            body=[\n                FunctionDef(\n                    name='mount',\n                    args=arguments(\n                        posonlyargs=[],\n                        args=[\n                            arg(arg='self'),\n                            arg(arg='subapp'),\n                            arg(arg='url_prefix')],\n                        kwonlyargs=[],\n                        kw_defaults=[],\n                        defaults=[\n                            Constant(value='')]),\n                    body=[\n                        Expr(\n                            value=Constant(value='Mount a sub-application, optionally under the given URL prefix.\\n        :param subapp: The sub-application to mount.\\n        :param url_prefix: The URL prefix to mount the application under.\\n        ')),\n                        For(\n                    